<a href="https://colab.research.google.com/github/athiranair846/ml-/blob/main/Copy_of_Untitled14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

# Upload dataset zip files (FER-2013.zip and RAVDESS.zip)
uploaded = files.upload()


Saving archive (2).zip to archive (2).zip
Saving archive (1).zip to archive (1).zip


In [ ]:
import zipfile
import os

# Extract FER-2013 dataset
fer_zip_path = "/content/archive (1).zip"
ravdess_zip_path = "/content/archive (2).zip"

with zipfile.ZipFile(fer_zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/facial_emotions")

with zipfile.ZipFile(ravdess_zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/voice_emotions")

print("Datasets extracted successfully!")

Datasets extracted successfully!


In [ ]:
os.makedirs("/content/models", exist_ok=True)
print("Model folder ready!")


Model folder ready!


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Path to extracted dataset
dataset_path = "/content/facial_emotions"

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(48,48),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(48,48),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 28711 images belonging to 2 classes.
Found 7176 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Get the number of classes from the training generator
num_classes = train_generator.num_classes

face_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax') # Use num_classes here
])

face_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
face_model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       819,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 838,402 (3.20 MB)

 Trainable params: 838,402 (3.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = face_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)


Epoch 1/5
898/898 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.7954 - loss: 0.5216

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


898/898 ━━━━━━━━━━━━━━━━━━━━ 95s 103ms/step - accuracy: 0.7954 - loss: 0.5216 - val_accuracy: 0.8000 - val_loss: 0.5017
Epoch 2/5
898/898 ━━━━━━━━━━━━━━━━━━━━ 85s 94ms/step - accuracy: 0.7979 - loss: 0.5083 - val_accuracy: 0.8000 - val_loss: 0.5004
Epoch 3/5
898/898 ━━━━━━━━━━━━━━━━━━━━ 85s 95ms/step - accuracy: 0.8004 - loss: 0.5036 - val_accuracy: 0.8000 - val_loss: 0.5008
Epoch 4/5
898/898 ━━━━━━━━━━━━━━━━━━━━ 84s 93ms/step - accuracy: 0.8019 - loss: 0.5002 - val_accuracy: 0.8000 - val_loss: 0.5006
Epoch 5/5
898/898 ━━━━━━━━━━━━━━━━━━━━ 89s 99ms/step - accuracy: 0.7993 - loss: 0.5019 - val_accuracy: 0.8000 - val_loss: 0.5012


In [ ]:
face_model.save("/content/models/facial_emotion_model.h5")
print("Facial emotion model saved!")


Facial emotion model saved!


In [ ]:
import librosa
import numpy as np
import os

voice_path = "/content/voice_emotions"
X, y = [], []

# Define a mapping from the emotion code in the filename to a label
# Based on RAVDESS documentation: 01=neutral, 02=calm, 03=happy, 04=sad, 05=angry, 06=fearful, 07=disgust, 08=surprised
emotion_mapping = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

def extract_mfcc(file_path, max_len=50):
    y_audio, sr = librosa.load(file_path, duration=3)
    mfcc = librosa.feature.mfcc(y=y_audio, sr=sr, n_mfcc=13)
    # Pad or truncate the MFCCs to a fixed length
    if mfcc.shape[1] < max_len:
        mfcc = np.pad(mfcc, ((0, 0), (0, max_len - mfcc.shape[1])), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc

# Iterate through actor folders
for actor_folder in os.listdir(voice_path):
    actor_path = os.path.join(voice_path, actor_folder)
    if os.path.isdir(actor_path): # Ensure it's a directory
        for file in os.listdir(actor_path):
            if file.endswith(".wav"):
                file_path = os.path.join(actor_path, file)
                # Extract emotion label from filename (e.g., 03-01-01-01-01-01-01.wav -> emotion code is the third part)
                emotion_code = file.split('-')[2]
                if emotion_code in emotion_mapping:
                    feature = extract_mfcc(file_path)
                    X.append(feature)
                    y.append(emotion_mapping[emotion_code])

X = np.array(X)
y = np.array(y)

print("Audio data shape:", X.shape, y.shape)

Audio data shape: (1440, 13, 50) (1440,)


In [ ]:
import os

# List the contents of the extracted voice emotions directory
print(os.listdir("/content/voice_emotions"))

['Actor_16', 'Actor_07', 'Actor_04', 'Actor_13', 'Actor_19', 'Actor_02', 'Actor_01', 'Actor_06', 'Actor_20', 'Actor_10', 'Actor_03', 'Actor_14', 'Actor_11', 'Actor_15', 'Actor_05', 'Actor_23', 'audio_speech_actors_01-24', 'Actor_22', 'Actor_09', 'Actor_24', 'Actor_17', 'Actor_21', 'Actor_08', 'Actor_18', 'Actor_12']


In [ ]:
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import LabelEncoder

# Encode string labels to integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Get the number of classes from the encoded labels
num_voice_classes = len(label_encoder.classes_)

voice_model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.3),
    LSTM(64),
    Dense(32, activation='relu'),
    Dense(num_voice_classes, activation='softmax')
])

voice_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
voice_model.summary()

voice_model.fit(X, y_encoded, epochs=5, batch_size=32, validation_split=0.2)
voice_model.save("/content/models/voice_stress_model.h5")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 13, 128)        │        91,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 13, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 143,400 (560.16 KB)

 Trainable params: 143,400 (560.16 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - accuracy: 0.1861 - loss: 2.0556 - val_accuracy: 0.2118 - val_loss: 2.0138
Epoch 2/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.2785 - loss: 1.9517 - val_accuracy: 0.1910 - val_loss: 1.9939
Epoch 3/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.3596 - loss: 1.7865 - val_accuracy: 0.2188 - val_loss: 2.0195
Epoch 4/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.3967 - loss: 1.6328 - val_accuracy: 0.2292 - val_loss: 2.0525
Epoch 5/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.5102 - loss: 1.4089 - val_accuracy: 0.2569 - val_loss: 2.1195


In [ ]:
from tensorflow.keras.models import load_model

# Load saved models
face_model = load_model("/content/models/facial_emotion_model.h5")
voice_model = load_model("/content/models/voice_stress_model.h5")

def combined_stress_score(face_emotion, voice_level):
    emotion_scores = {'Happy':0, 'Neutral':1, 'Sad':2, 'Angry':3}
    total_score = emotion_scores.get(face_emotion, 1) + voice_level
    if total_score <= 2:
        return "Low Stress"
    elif total_score <= 4:
        return "Medium Stress"
    else:
        return "High Stress"

# Example test
predicted_face_emotion = "Angry"
predicted_voice_level = 2  # 0=calm,1=stressed,2=highly stressed

final_stress = combined_stress_score(predicted_face_emotion, predicted_voice_level)
print("Final Stress Level:", final_stress)


Final Stress Level: High Stress


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the trained facial emotion model
face_model = load_model("/content/models/facial_emotion_model.h5")

# Function to predict emotion
def predict_emotion(img_path):
    img = image.load_img(img_path, target_size=(48,48), color_mode="grayscale")
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = face_model.predict(img_array)
    emotion_classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
    return emotion_classes[np.argmax(prediction)]

# Upload an image to test
from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
    emotion = predict_emotion(filename)
    print(f"Detected Facial Emotion: {emotion}")


KeyboardInterrupt: 

In [ ]:
# Load facial emotion model
facial_model = load_model('/content/models/facial_emotion_model.h5')

# Load voice stress model
voice_model = load_model('/content/models/voice_stress_model.h5')

# No need to compile if only predicting
print("Models loaded successfully!")


Models loaded successfully!


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from scipy.io import wavfile
import matplotlib.pyplot as plt
from google.colab import files
import librosa
import librosa.display
import io
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

def predict_facial_emotion(image_path):
    img = image.load_img(image_path, target_size=(48, 48), color_mode="grayscale")
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    prediction = facial_model.predict(img_array)
    predicted_index = np.argmax(prediction)
    predicted_emotion = emotion_labels[predicted_index]

    return predicted_emotion


In [ ]:
# Define a mapping from the emotion code in the filename to a label
# Based on RAVDESS documentation: 01=neutral, 02=calm, 03=happy, 04=sad, 05=angry, 06=fearful, 07=disgust, 08=surprised
voice_labels = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']

def predict_voice_stress(audio_path):
    # Load the audio file
    y, sr = librosa.load(audio_path, duration=3, offset=0.5)

    # Extract MFCC features
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13) # Use n_mfcc=13 to match training data
    # Pad or truncate the MFCCs to a fixed length
    max_len = 50 # Use the same max_len as in training
    if mfcc.shape[1] < max_len:
        mfcc = np.pad(mfcc, ((0, 0), (0, max_len - mfcc.shape[1])), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]

    # Reshape for LSTM input (batch_size, timesteps, features)
    mfcc_reshaped = np.expand_dims(mfcc, axis=0)


    # Predict
    prediction = voice_model.predict(mfcc_reshaped)
    predicted_index = np.argmax(prediction)
    predicted_stress = voice_labels[predicted_index]

    return predicted_stress

In [ ]:
def combined_stress_score(facial_emotion, voice_stress):
    if facial_emotion in ['Angry', 'Fear', 'Sad'] and voice_stress == 'Highly_Stressed':
        return "High Stress"
    elif voice_stress == 'Highly_Stressed':
        return "Moderate Stress"
    else:
        return "Low Stress"


In [ ]:
uploaded = files.upload()

for file_name in uploaded.keys():
    facial_result = predict_facial_emotion(file_name)
    print(f"Detected Facial Emotion: {facial_result}")


Saving 360_F_703430862_kFTAoTzCriebp5dBZX99g3FEz2omUmm5.jpg to 360_F_703430862_kFTAoTzCriebp5dBZX99g3FEz2omUmm5.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
Detected Facial Emotion: Disgust


In [ ]:
uploaded = files.upload()

for file_name in uploaded.keys():
    # Check if the uploaded file is an audio file (e.g., .wav)
    if file_name.lower().endswith('.wav'):
        voice_result = predict_voice_stress(file_name)
        print(f"Detected Voice Stress Level: {voice_result}")
    else:
        print(f"Skipping non-audio file: {file_name}. Please upload a .wav file.")

Saving harvard.wav to harvard (1).wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Detected Voice Stress Level: disgust


In [ ]:
final_result = combined_stress_score(facial_result, voice_result)
print("\nFINAL OVERALL STRESS LEVEL:", final_result)



FINAL OVERALL STRESS LEVEL: Low Stress


In [ ]:
from google.colab import files

# Upload multiple face and voice files together
uploaded = files.upload()


Saving 360_F_197116416_hpfTtXSoJMvMqU99n6hGP4xX0ejYa4M7.jpg to 360_F_197116416_hpfTtXSoJMvMqU99n6hGP4xX0ejYa4M7 (2).jpg
Saving 360_F_703430862_kFTAoTzCriebp5dBZX99g3FEz2omUmm5.jpg to 360_F_703430862_kFTAoTzCriebp5dBZX99g3FEz2omUmm5 (3).jpg
Saving addf8-Alaw-GW.wav to addf8-Alaw-GW.wav
Saving harvard.wav to harvard (2).wav
Saving M1F1-Alaw-AFsp.wav to M1F1-Alaw-AFsp.wav
Saving portrait-white-man-isolated_53876-40306.jpg to portrait-white-man-isolated_53876-40306 (1).jpg


In [ ]:
import os

face_files = []
voice_files = []

for file_name in uploaded.keys():
    if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        face_files.append(file_name)
    elif file_name.lower().endswith('.wav'):
        voice_files.append(file_name)

print("Face files uploaded:", face_files)
print("Voice files uploaded:", voice_files)


Face files uploaded: ['360_F_197116416_hpfTtXSoJMvMqU99n6hGP4xX0ejYa4M7 (2).jpg', '360_F_703430862_kFTAoTzCriebp5dBZX99g3FEz2omUmm5 (3).jpg', 'portrait-white-man-isolated_53876-40306 (1).jpg']
Voice files uploaded: ['addf8-Alaw-GW.wav', 'harvard (2).wav', 'M1F1-Alaw-AFsp.wav']


In [ ]:
import random

# Pick random test files
random_face = random.choice(face_files)
random_voice = random.choice(voice_files)

print("Selected Face File:", random_face)
print("Selected Voice File:", random_voice)


Selected Face File: portrait-white-man-isolated_53876-40306 (1).jpg
Selected Voice File: harvard (2).wav


In [ ]:
# Predict facial emotion
facial_result = predict_facial_emotion(random_face)
print(f"Detected Facial Emotion: {facial_result}")

# Predict voice stress
voice_result = predict_voice_stress(random_voice)
print(f"Detected Voice Stress Level: {voice_result}")

# Combine both
final_result = combined_stress_score(facial_result, voice_result)
print("\nFINAL OVERALL STRESS LEVEL:", final_result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Detected Facial Emotion: Disgust
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Detected Voice Stress Level: disgust

FINAL OVERALL STRESS LEVEL: Low Stress


In [ ]:
for i in range(5):  # Run 5 random tests
    random_face = random.choice(face_files)
    random_voice = random.choice(voice_files)

    facial_result = predict_facial_emotion(random_face)
    voice_result = predict_voice_stress(random_voice)
    final_result = combined_stress_score(facial_result, voice_result)

    print(f"\nTest Case {i+1}")
    print(f"Face: {random_face} -> {facial_result}")
    print(f"Voice: {random_voice} -> {voice_result}")
    print(f"Overall Stress Level: {final_result}")
    print("-" * 40)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step

Test Case 1
Face: 360_F_197116416_hpfTtXSoJMvMqU99n6hGP4xX0ejYa4M7 (2).jpg -> Disgust
Voice: M1F1-Alaw-AFsp.wav -> sad
Overall Stress Level: Low Stress
----------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step

Test Case 2
Face: 360_F_197116416_hpfTtXSoJMvMqU99n6hGP4xX0ejYa4M7 (2).jpg -> Disgust
Voice: harvard (2).wav -> disgust
Overall Stress Level: Low Stress
----------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step

Test Case 3
Face: portrait-white-man-isolated_53876-40306 (1).jpg -> Disgust
Voice: M1F1-Alaw-AFsp.wav -> sad
Overall Stress Level: Low Stress
----------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step

Test Case 4
Face: portrait-white-man-isolated_53876-40306 (1).jpg -> Disgust
Voice: M1F1-Alaw-AFsp.wav -> sad
Overall 